#### 개요
학습대상 레코드와 컬러으로 구성된 데이터를 대상으로 배치/온라인 전처리 수행

##### 주요 작업
* 공사비 전처리(NaN, 컬럼추가)
* 전주/전선/인입선 갯 수 계산
* 데이터 전처리 및 공사비 데이터에 추가
  * 전주/전선/인입선 순으로 진행

##### 향후 추가할 사항(옵션)
* 전주/전선에 대한 순서 지정(좌표정보 처리)
* 사번 데이터 문자 정보 추가(중요도 하)
* 향후 작업이 필요할 경우 아래 코드 참조
  * aidd-back240313/batch/preprocessing.py

In [1]:
import re
import pandas as pd

import aidd.sys.config as cfg
from aidd.utils.data_io import save_data
from aidd.utils.data_io import read_pickle, save_pickle

# 이 부분은 IPYNB에서 작업할 경우 만 필요함
# * IPYNB에서 merge.ipynb와 preprocessing.ipynb가 끈어져 추가한 부분임
from aidd.utils.data_io import get_merged_data

#### 병합 데이터 불러오기

In [2]:
bdf_dict = get_merged_data()
odf_dict = get_merged_data(mode='ONLINE', dtype={'CONS_ID': str})

Data Type: CONS, Size: (15335, 8), pTime: 0:00:00.030596
Data Type: POLE, Size: (27896, 6), pTime: 0:00:00.035545
Data Type: LINE, Size: (30348, 11), pTime: 0:00:00.075465
Data Type: SL, Size: (17034, 6), pTime: 0:00:00.013458
Data Type: CONS, Size: (3, 8), pTime: 0:00:00.001446
Data Type: POLE, Size: (9, 6), pTime: 0:00:00.001036
Data Type: LINE, Size: (9, 11), pTime: 0:00:00.001318
Data Type: SL, Size: (3, 6), pTime: 0:00:00.000929


In [3]:
_df = odf_dict['POLE']
_df.columns

Index(['CONS_ID', 'COMP_ID', 'POLE_SHAPE_CD', 'POLE_TYPE_CD', 'POLE_SPEC_CD',
       'COORDINATE'],
      dtype='object')

#### 전처리

##### 공사비

In [4]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def cons(df_dict=bdf_dict, is_batch=True):
    df = df_dict['CONS']
    print(f'최초 공사비 데이터 크기: {df.shape}')
    
    # 결측치 처리
    df.fillna(0, inplace=True)
        
    # 일자정보 처리
    # * '최종변경일시'를 이용해 다양한 일자정보 컬럼 추가
    # * 참고로 일자정보가 날자형식이 아니면 날자형식으로 변환
    if df.LAST_MOD_DATE.dtype != '<M8[ns]':
        df.LAST_MOD_DATE = pd.to_datetime(df.LAST_MOD_DATE)
    df['YEAR'] = df.LAST_MOD_DATE.dt.year
    df['MONTH'] = df.LAST_MOD_DATE.dt.month
    df['DAY'] = df.LAST_MOD_DATE.dt.day
    df['DAYOFWEEK'] = df.LAST_MOD_DATE.dt.dayofweek
    df['DAYOFYEAR'] = df.LAST_MOD_DATE.dt.dayofyear
    df['YEAR_MONTH'] = df.LAST_MOD_DATE.dt.strftime("%Y%m").astype(int)
    
    # 사번정보 처리
    # '최종변경자사번(LAST_MOD_EID)'만 사용(최초등록자==최종변경자사번)
    df['EID_NUMBER'] = df.LAST_MOD_EID.apply(
        lambda x: re.findall(r'\d+', x)[0]
    ).astype(int)
    
    # 사업소명 숫자로 변환
    # 모델학습 만을 생각하면 rank()함수를 사용할 수 있지만, 
    # rank를 사용하면 서비스시에 같은 값 다른 ranking을 제공하기 때문에 문제 발생
    # 이를 해결하기 위해 학습시 사업소명을 저장하고 
    # 서비스시 해당 사업소명리스트를 이용해 숫자로 변경해줘야 함.
    if is_batch:
        offc_list = df.OFFICE_NAME.unique().tolist()
        save_pickle(offc_list, file_code='DUMP,OFFICE_LIST')
    else:
        offc_list = read_pickle(file_code='DUMP,OFFICE_LIST')
    offc_idxs = []
    for oname in df.OFFICE_NAME:
        offc_idxs.append(offc_list.index(oname)) 
    df['OFFICE_NUMBER'] = offc_idxs
    
    # 필요 컬럼 추출
    # 'CONT_CAP', 'EID_CODE_NUMBER' 추가 필요
    df = df[cfg.COLS['PP']['CONS']['PP']]
    print(f'컬럼 추가 후 공사비 데이터 크기: {df.shape}')
    
    if is_batch:
        save_data(df, file_code='PP,CONS')
    
    return df

In [5]:
pb_df = cons()

최초 공사비 데이터 크기: (15335, 8)
컬럼 추가 후 공사비 데이터 크기: (15335, 14)


In [6]:
pb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15335 entries, 0 to 15334
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          15335 non-null  object        
 1   TOTAL_CONS_COST  15335 non-null  int64         
 2   LAST_MOD_DATE    15335 non-null  datetime64[ns]
 3   LAST_MOD_EID     15335 non-null  object        
 4   OFFICE_NAME      15335 non-null  object        
 5   CONT_CAP         15335 non-null  int64         
 6   YEAR             15335 non-null  int32         
 7   MONTH            15335 non-null  int32         
 8   DAY              15335 non-null  int32         
 9   DAYOFWEEK        15335 non-null  int32         
 10  DAYOFYEAR        15335 non-null  int32         
 11  YEAR_MONTH       15335 non-null  int64         
 12  EID_NUMBER       15335 non-null  int64         
 13  OFFICE_NUMBER    15335 non-null  int64         
dtypes: datetime64[ns](1), int32(5), int64(

In [7]:
po_df = cons(df_dict=odf_dict, is_batch=False)

최초 공사비 데이터 크기: (3, 8)
컬럼 추가 후 공사비 데이터 크기: (3, 14)


In [8]:
po_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   CONS_ID          3 non-null      object        
 1   TOTAL_CONS_COST  3 non-null      int64         
 2   LAST_MOD_DATE    3 non-null      datetime64[ns]
 3   LAST_MOD_EID     3 non-null      object        
 4   OFFICE_NAME      3 non-null      object        
 5   CONT_CAP         3 non-null      int64         
 6   YEAR             3 non-null      int32         
 7   MONTH            3 non-null      int32         
 8   DAY              3 non-null      int32         
 9   DAYOFWEEK        3 non-null      int32         
 10  DAYOFYEAR        3 non-null      int32         
 11  YEAR_MONTH       3 non-null      int64         
 12  EID_NUMBER       3 non-null      int64         
 13  OFFICE_NUMBER    3 non-null      int64         
dtypes: datetime64[ns](1), int32(5), int64(5), obje

##### 설비 갯 수

In [9]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def compute_and_check_facilities_count(
    sdf=pb_df, df_dict=bdf_dict, is_batch=True):
    
    # 공사비 데이터에 카운트 컬럼 추가
    mdf = sdf
    # 공사번호별 설비 갯 수 계산
    for key in cfg.DATA_SETs[1:]:
        df = df_dict[key]
        count_cons_ids = df.CONS_ID.value_counts()
        col_name = f'{key}_CNT'
        mdf = pd.merge(
            mdf, count_cons_ids.rename(col_name),
            left_on='CONS_ID', right_on=count_cons_ids.index, how='left'
        )
        # 해당 공사번호에 없는 설비는 NaN처리되며, 이 값을 0으로 변경
        mdf[col_name] = mdf[col_name].fillna(0)  
    print(f'설비 카운트가 추가된 공사비 데이터 크기: {mdf.shape}')
    
    # 모델 학습에 사용할 레코드 추출
    # * 전주/전선 갯 수가 10개 이상인 경우 
    modeling_records = \
        (mdf.POLE_CNT >= cfg.CONSTRAINTS['MIN_POLE_CNT']) & \
        (mdf.POLE_CNT <= cfg.CONSTRAINTS['MAX_POLE_CNT']) & \
        (mdf.LINE_CNT >= cfg.CONSTRAINTS['MIN_LINE_CNT']) & \
        (mdf.LINE_CNT <= cfg.CONSTRAINTS['MAX_LINE_CNT'])
    mdf = mdf[modeling_records].reset_index(drop=True)
    print(f'제약조건이 적용된 공사비 데이터 크기: {mdf.shape}')
    
    if is_batch:
        save_data(mdf, file_code='PP,CONS')
    return mdf

In [10]:
pb_df = compute_and_check_facilities_count()

설비 카운트가 추가된 공사비 데이터 크기: (15335, 17)
제약조건이 적용된 공사비 데이터 크기: (14729, 17)


In [11]:
po_df = compute_and_check_facilities_count(
    sdf=po_df, df_dict=odf_dict, is_batch=False
)

설비 카운트가 추가된 공사비 데이터 크기: (3, 17)
제약조건이 적용된 공사비 데이터 크기: (3, 17)


In [12]:
po_df

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,CONT_CAP,YEAR,MONTH,DAY,DAYOFWEEK,DAYOFYEAR,YEAR_MONTH,EID_NUMBER,OFFICE_NUMBER,POLE_CNT,LINE_CNT,SL_CNT
0,477420204194,9076645,2020-12-23 11:50:39,MDE1900011,단양지사,3,2020,12,23,2,358,202012,1900011,0,4,4,1.0
1,475920223725,2176378,2022-11-24 10:42:22,MDP2100032,음성지사,3,2022,11,24,3,328,202211,2100032,5,1,1,0.0
2,474620226651,9512744,2023-04-04 08:27:05,MDP2300252,충주지사,5,2023,4,4,1,94,202304,2300252,10,4,4,2.0


In [13]:
pb_df[pb_df.CONS_ID.isin(po_df.CONS_ID)]

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,CONT_CAP,YEAR,MONTH,DAY,DAYOFWEEK,DAYOFYEAR,YEAR_MONTH,EID_NUMBER,OFFICE_NUMBER,POLE_CNT,LINE_CNT,SL_CNT
75,477420204194,9076645,2020-12-23 11:50:39,MDE1900011,단양지사,3,2020,12,23,2,358,202012,1900011,0,4.0,4.0,1.0
7056,475920223725,2176378,2022-11-24 10:42:22,MDP2100032,음성지사,3,2022,11,24,3,328,202211,2100032,5,1.0,1.0,0.0
14551,474620226651,9512744,2023-04-04 08:27:05,MDP2300252,충주지사,5,2023,4,4,1,94,202304,2300252,10,4.0,4.0,2.0


##### 전주

In [14]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def pole(sdf=pb_df, df_dict=bdf_dict, is_batch=True):
    df = df_dict['POLE']
    print(f'최초 전주 데이터 크기: {df.shape}')
    
    # 결측치 처리
    df.fillna(0, inplace=True)
    
    # 코드형 컬럼 One-Hot Encoding
    prefix = ['POLE_SHAPE', 'POLE_TYPE', 'POLE_SPEC']
    columns = [x+'_CD' for x in prefix]
    # 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
    # (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
    if df.POLE_SPEC_CD.dtype != 'float64':
        df['POLE_SPEC_CD'] = df['POLE_SPEC_CD'].astype(float)
    df = pd.get_dummies(df, columns=columns, prefix=prefix)
    # True, False값을 1, 0으로 변환
    df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
    
    # 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나온 컬럼 리스트 저장
    df_cols = df.columns.tolist()
    if is_batch:
        save_pickle(df_cols, file_code='DUMP,POLE_ONE_HOT_COLS')
    else:
        # 학습 당시 컬럼 불러오기
        modeling_cols = read_pickle(file_code='DUMP,POLE_ONE_HOT_COLS')
        # 실시간 처리에서 만들어 지지 않는 컬럼 추출
        append_cols = [x for x in modeling_cols if x not in df_cols]
        # 0으로 컬럼값 추가
        df.loc[:, append_cols] = 0
    print(f'One-Hot Encoding 결과: {df.shape}')
    
    # 공사비별 전주 데이터 합산
    unique_cons_ids = df.CONS_ID.unique()
    cons_id_pole_sums = []
    # 합산대상 컬럼 리스트 추출
    sum_cols = [col for col in df.columns if col.startswith('POLE_')]
    # 공사번호별 합산(시간이 좀 걸림, 14700건 처리에 약 40초 소요)
    for cid in unique_cons_ids:
        cons_id_pole_sums.append(
            [cid]+df[df.CONS_ID==cid][sum_cols].sum().values.tolist())
    # 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
    pole_sums_df = pd.DataFrame(
        cons_id_pole_sums, columns=['CONS_ID'] + sum_cols)
    
    # 공사비 데이터와 전주정보 그룹 데이터 병합
    pdf = pd.merge(
        sdf, pole_sums_df,
        left_on='CONS_ID', right_on='CONS_ID', how='left')
    print(f'전주 그룹정보가 적용된 데이터 크기: {pdf.shape}')
    
    if is_batch:
        save_data(pdf, file_code='PP,POLE')
    return pdf

In [15]:
pb_df = pole()

최초 전주 데이터 크기: (27896, 6)
One-Hot Encoding 결과: (27896, 22)


전주 그룹정보가 적용된 데이터 크기: (14729, 36)


In [16]:
po_df = pole(sdf=po_df, df_dict=odf_dict, is_batch=False)

최초 전주 데이터 크기: (9, 6)
One-Hot Encoding 결과: (9, 22)
전주 그룹정보가 적용된 데이터 크기: (3, 36)


##### 전선

In [17]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def line(sdf=pb_df, df_dict=bdf_dict, is_batch=True):
    df = df_dict['LINE']
    print(f'최초 전선 데이터 크기: {df.shape}')
    
    # 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
    # (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
    if df.LINE_SPEC_CD.dtype != 'float64':
        df['LINE_SPEC_CD'] = df['LINE_SPEC_CD'].astype(float)
    if df.NEUTRAL_SPEC_CD.dtype != 'float64':
        df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].astype(float)   
    # 중성선규격코드(NEUTRAL_SPEC_CD)에 0.0과 NaN이 존재(NaN=>999.0 변환)
    df['NEUTRAL_SPEC_CD'] = df['NEUTRAL_SPEC_CD'].fillna(999.0)
    # 중성선종류코드(NEUTRAL_TYPE_CD)의 NaN값을 문자열 'NaN'으로 치환
    df.NEUTRAL_TYPE_CD = df.NEUTRAL_TYPE_CD.fillna('NaN')
    # 결선방식이 41인 값이 1개만 존재하기 때문에 많이 있는 43으로 치환
    df.WIRING_SCHEME = df.WIRING_SCHEME.replace(41, 43)
    # 전선 전체길이 추가: = 선로길이(SPAN) * 전선 갯 수(PHASE)
    df.loc[:, 'LINE_LENGTH'] = df.SPAN * df.LINE_PHASE_CD

    # 결측치 처리
    df.fillna(0, inplace=True)
    
    # 코드형 컬럼 One-Hot Encoding
    # WIRING_SCHEME은 마지막에 '_CD'가 붙지 않음
    prefix = ['WIRING_SCHEME', 'LINE_TYPE', 'LINE_SPEC', 'LINE_PHASE',
              'NEUTRAL_TYPE', 'NEUTRAL_SPEC']
    columns = [x+'_CD' for x in prefix if x != 'WIRING_SCHEME']
    columns += ['WIRING_SCHEME']
    df = pd.get_dummies(df, columns=columns, prefix=prefix)
    # True, False를 1, 0으로 변환
    df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
    # 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나올 컬럼리스트 저장
    df_cols = df.columns.tolist()
    if is_batch:
        save_pickle(df_cols, file_code='DUMP,LINE_ONE_HOT_COLS')
    else:
        modeling_cols = read_pickle('DUMP,LINE_ONE_HOT_COLS')
        append_cols = [col for col in modeling_cols if col not in df_cols]
        df.loc[:, append_cols] = 0
    print(f'전선 One-Hot Encoding 결과: {df.shape}')
    
    # 공사비별 전선 데이터 합산
    unique_cons_ids = df.CONS_ID.unique()
    cons_id_line_sums = []
    sum_cols = ['SPAN'] + df.columns.tolist()[5:]
    for cid in unique_cons_ids:
        cons_id_line_sums.append(
            [cid]+df[df.CONS_ID==cid][sum_cols].sum().values.tolist())
    # 공사번호별로 합산된 전주 정보를 데이터프레임으로 변환
    line_sums_df = pd.DataFrame(
        cons_id_line_sums, columns=['CONS_ID']+sum_cols)
    
    # 공사비 데이터와 전주 그룹 데이터 병합
    pdf = pd.merge(
        sdf, line_sums_df,
        left_on='CONS_ID', right_on='CONS_ID', how='left')
    print(f'전선 그룹정보가 적용된 데이터 크기: {pdf.shape}')
    
    if is_batch:
        save_data(pdf, file_code='PP,LINE')
    return pdf

In [18]:
pb_df = line()
po_df = line(po_df, df_dict=odf_dict, is_batch=False)

최초 전선 데이터 크기: (30348, 11)
전선 One-Hot Encoding 결과: (30348, 48)


전선 그룹정보가 적용된 데이터 크기: (14729, 80)
최초 전선 데이터 크기: (9, 11)
전선 One-Hot Encoding 결과: (9, 48)
전선 그룹정보가 적용된 데이터 크기: (3, 80)


##### 인입선

In [19]:
# 이후 클래스 작성 시 함수의 파라메터와 리턴값은 클래스의 전역변수로 변환
def sl(sdf=pb_df, df_dict=bdf_dict, is_batch=True):
    df = df_dict['SL']
    print(f'최초 인입선 데이터 크기: {df.shape}')
    
    # 숫자형 값 통일(실수형이 아닌 값을 실수형으로 변환)
    # (One-Hot Encoding시 동일한 컬럼값을 만들기 위해 실행)
    if df.SL_SPEC_CD.dtype != 'float64':
        df['SL_SPEC_CD'] = df['SL_SPEC_CD'].astype(float)
    # 결측치 처리
    df.fillna(0, inplace=True)
    
    # 코드형 컬럼 One-Hot Encoding
    prefix = ['SL_TYPE', 'SL_SPEC']
    columns = [col+'_CD' for col in prefix]
    df = pd.get_dummies(df, columns=columns, prefix=prefix)
    df = df.apply(lambda x: int(x) if isinstance(x, bool) else x)
    # 실시간 처리에서 동일 컬럼을 추가하기 위해 학습에서 나올 컬럼리스트 저장
    df_cols = df.columns.tolist()
    if is_batch:
        save_pickle(df_cols, file_code='DUMP,SL_ONE_HOT_COLS')
    else:
        modeling_cols = read_pickle('DUMP,SL_ONE_HOT_COLS')
        append_cols = [col for col in modeling_cols if col not in df_cols]
        df.loc[:, append_cols] = 0
    print(f'인입선 One-Hot Encoding 결과: {df.shape}')
    
    # 공사비별 인입선 데이터 합산
    unique_cons_ids = df.CONS_ID.unique()
    cons_id_sl_sums = []
    sum_cols = df.columns.tolist()[2:]
    for cid in unique_cons_ids:
        _df = df[df.CONS_ID==cid]
        sl_sums = _df[sum_cols].sum().values.tolist()
        sl_comp_id_cnt = _df.COMP_ID.nunique()
        cons_id_sl_sums.append(
            [cid, sl_comp_id_cnt, _df.shape[0]] + sl_sums)
    # 데이터프레임 만들기
    sl_sums_df = pd.DataFrame(
        cons_id_sl_sums, 
        columns=['CONS_ID', 'SL_COMP_ID_CNT', 'REAL_SL_CNT', 'SL_SPAN_SUM'] \
              + sum_cols[1:]
    )
    
    # 공사비 데이터와 인입선 그룹 데이터 병합
    pdf = pd.merge(
        sdf, sl_sums_df,
        left_on='CONS_ID', right_on='CONS_ID', how='left'
    )
    print(f'인입선 그룹정보가 적용된 데이터 크기: {pdf.shape}')
    
    # 최종 완료시점에서 NaN값을 0으로 처리
    # 온라인 작업 시 인입선이 없거나 전주가 없는 작업 등에서 NaN가 올 수 있음
    pdf.fillna(0, inplace=True)
    # 모델링 시점과 서비스 시점의 데이터프레임 순서를 동일하게 하기 위해
    # 모델링 시점의 컬럼 순서를 저장해 서비스 시점에서 컬럼 순서를 재배치
    # One-Hot Encoding시점에 데이터 컬럼의 순서가 변경될 수 있음.
    if is_batch:
        last_pp_cols = pdf.columns
        save_pickle(last_pp_cols, file_code='DUMP,LAST_PP_COLS')
    else:
        last_pp_cols = read_pickle('DUMP,LAST_PP_COLS')
        pdf = pdf.reindex(columns=last_pp_cols)
        
    if is_batch:
        save_data(pdf, file_code='PP,SL')
    return pdf
    

In [20]:
pb_df = sl()
po_df = sl(po_df, df_dict=odf_dict, is_batch=False)

최초 인입선 데이터 크기: (17034, 6)
인입선 One-Hot Encoding 결과: (17034, 30)
인입선 그룹정보가 적용된 데이터 크기: (14729, 110)
최초 인입선 데이터 크기: (3, 6)
인입선 One-Hot Encoding 결과: (3, 30)
인입선 그룹정보가 적용된 데이터 크기: (3, 110)


In [21]:
# 온라인 시험용 최종 전처리 데이터 저장
save_data(po_df, file_code='ONLINE')

In [22]:
po_df

,CONS_ID,TOTAL_CONS_COST,LAST_MOD_DATE,LAST_MOD_EID,OFFICE_NAME,CONT_CAP,YEAR,MONTH,DAY,DAYOFWEEK,...,SL_SPEC_16.0,SL_SPEC_22.0,SL_SPEC_25.0,SL_SPEC_35.0,SL_SPEC_38.0,SL_SPEC_60.0,SL_SPEC_70.0,SL_SPEC_100.0,SL_SPEC_120.0,SL_SPEC_240.0
0,477420204194,9076645,2020-12-23 11:50:39,MDE1900011,단양지사,3,2020,12,23,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,475920223725,2176378,2022-11-24 10:42:22,MDP2100032,음성지사,3,2022,11,24,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,474620226651,9512744,2023-04-04 08:27:05,MDP2300252,충주지사,5,2023,4,4,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
po_df.columns

Index(['CONS_ID', 'TOTAL_CONS_COST', 'LAST_MOD_DATE', 'LAST_MOD_EID',
       'OFFICE_NAME', 'CONT_CAP', 'YEAR', 'MONTH', 'DAY', 'DAYOFWEEK',
       ...
       'SL_SPEC_16.0', 'SL_SPEC_22.0', 'SL_SPEC_25.0', 'SL_SPEC_35.0',
       'SL_SPEC_38.0', 'SL_SPEC_60.0', 'SL_SPEC_70.0', 'SL_SPEC_100.0',
       'SL_SPEC_120.0', 'SL_SPEC_240.0'],
      dtype='object', length=110)